In [1]:
import numpy as np
import pandas as pd
import torch
from scipy.linalg import kron
from scipy.sparse.linalg import svds

In [2]:
from numba import jit

In [3]:
from numba import cuda

In [4]:
import numpy as np
import pandas as pd
import torch
from scipy.linalg import kron
from scipy.sparse.linalg import svds

import pickle
with open("list_of_grads1", "rb") as fp:   #Pickling
    list_of_grads = pickle.load(fp)

list_of_grads = torch.stack(list_of_grads, dim=0)

grad_vectors = torch.stack([grad.T.reshape(-1) for grad in list_of_grads])

In [5]:
list_of_grads.shape

torch.Size([625, 192, 768])

In [9]:
list_of_grads[10]

tensor([[ 0.0007,  0.0016,  0.0000,  ...,  0.0044, -0.0017, -0.0002],
        [ 0.0016,  0.0045,  0.0000,  ...,  0.0030, -0.0043,  0.0024],
        [-0.0015, -0.0033,  0.0000,  ..., -0.0031, -0.0017, -0.0003],
        ...,
        [ 0.0003, -0.0004,  0.0000,  ..., -0.0003, -0.0010, -0.0001],
        [ 0.0122,  0.0169,  0.0000,  ...,  0.0046, -0.0025,  0.0029],
        [-0.0028, -0.0018,  0.0000,  ..., -0.0017,  0.0068,  0.0002]])

In [10]:
m = list_of_grads[0].shape[0]
n = list_of_grads[0].shape[1]


In [11]:
grad_vectors = torch.stack([grad.T.reshape(-1) for grad in list_of_grads])

In [ ]:
import numba

print (m, n)

from scipy.sparse.linalg import svds, LinearOperator


grad_vectors = torch.stack([grad.T.reshape(-1) for grad in list_of_grads])
print (grad_vectors.shape)
d_grad_vectors = cuda.to_device(grad_vectors)

reduce_init_val = 0.0


@cuda.jit(device=True)
def getitem_fmatrices_permuted(d_grad_vectors, p, q):
    reduce_init_val = 0.0
    for idx in range(d_grad_vectors.shape[0]):
        #reduce_init_val+= d_grad_vectors[idx][p]*d_grad_vectors[idx][q]
        elem = d_grad_vectors[idx]
        reduce_init_val += elem[p] * elem[q]

    return reduce_init_val/d_grad_vectors.shape[0]

@cuda.jit
def matvec_kernel(d_grad_vectors, x, res, m, n):
    """
    CUDA kernel to compute matvec operation (A * x).
    """
    idx = cuda.grid(1)
    
    pd, pn = divmod(np.int32(idx), m)
    if idx < res.size:  # Each thread computes one element of the result
        sum_ = 0.0
        for jnd in range(x.size):
            qd, qn = divmod(jnd, n)
            sum_ += x[jnd]*getitem_fmatrices_permuted(d_grad_vectors, (pd)*n +qd, (pn)*n + qn)#d_grad_vector[(pd)*n +qd]*d_grad_vector[(pn)*n + qn] 
        res[idx] = sum_

@cuda.jit
def rmatvec_kernel(d_grad_vectors, x, res, m, n):
    """
    CUDA kernel to compute rmatvec operation (A.T * x).
    """
    idx = cuda.grid(1)
    if idx < res.size:  # Each thread computes one element of the result
        sum_ = 0.0
        pd, pn = divmod(np.int32(idx), n)
        for jnd in range(x.size):
            jd, jn = divmod(jnd, m)
            #jnd, idx
            sum_ += x[jnd]*getitem_fmatrices_permuted(d_grad_vectors, jd*n +pd, jn*n + pn)# d_grad_vector[(jnd//m)*n +pd]*d_grad_vector[(jnd%m)*n + pn]  #grad_vector[(q // m)*n +p // n]*grad_vector[(q % m)*n + p % n]
        res[idx] = sum_


def matvec_item_custom(x):
    """
    Host function to launch matvec_kernel.
    """
    print ("matvec")
    x = x.ravel()
    res = np.zeros((m**2), dtype=np.float32)
    threads_per_block = 1024
    blocks_per_grid = (m**2 + threads_per_block - 1) // threads_per_block
    print (m**2, threads_per_block, blocks_per_grid)


    #d_grad_vector = cuda.to_device(grad_vector)
    d_x = cuda.to_device(x)
    d_res = cuda.device_array((m**2), dtype=np.float32)

    matvec_kernel[blocks_per_grid, threads_per_block](d_grad_vectors, d_x, d_res, m, n)

    # Copy result back to host
    res = d_res.copy_to_host()
    print (res)
    return res

def rmatvec_item_custom(x):
    """
    Host function to launch rmatvec_kernel.
    """
    res = np.zeros((n**2), dtype=np.float32)
    print ("rmatvec")
    x = x.ravel()
    threads_per_block = 1024
    blocks_per_grid = (n**2 + threads_per_block - 1) // threads_per_block


    #d_grad_vector = cuda.to_device(grad_vector)
    d_x = cuda.to_device(x)
    d_res = cuda.device_array((n**2), dtype=np.float32)

    rmatvec_kernel[blocks_per_grid, threads_per_block](d_grad_vectors, d_x, d_res, m, n)

    # Copy result back to host
    res = d_res.copy_to_host()
    return res


linop_v = LinearOperator(
    shape=(m**2, n**2),
    matvec=matvec_item_custom,
    rmatvec=rmatvec_item_custom
)
print("Operator created", flush=True)

# Compute SVD
print("Performing SVD...", flush=True)
v, s, u = svds(linop_v, k = 3, return_singular_vectors=True)
s = np.sort(s)[::-1]

192 768
torch.Size([625, 147456])
matvec
36864 1024 36


/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 36 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[0. 0. 0. ... 0. 0. 0.]
Operator created
Performing SVD...
rmatvec
matvec
36864 1024 36


In [18]:
s

array([0.5209037 , 0.22825046, 0.22730228], dtype=float32)

In [19]:
C1 = s[0] * u[-1, :].reshape(n, n)
B1 = v[:, -1].reshape(m, m)
#np.li

In [20]:
C1

array([[-2.2992203e-03, -2.0863640e-03,  5.2899041e-04, ...,
         2.1026177e-05, -7.7487185e-04,  9.5214468e-04],
       [-2.0864853e-03, -3.1914932e-03,  6.1957870e-04, ...,
         1.9511606e-05, -4.8029647e-04,  6.2608899e-04],
       [ 5.2901125e-04,  6.1957882e-04, -3.4995459e-04, ...,
         6.6825905e-06,  3.0172875e-04, -2.5753380e-04],
       ...,
       [ 2.1026184e-05,  1.9511606e-05,  6.6825919e-06, ...,
        -2.0335838e-05,  3.5387606e-05,  9.9923554e-06],
       [-7.7487197e-04, -4.8029659e-04,  3.0172878e-04, ...,
         3.5387602e-05, -3.9137211e-03,  1.3470693e-03],
       [ 9.5214479e-04,  6.2608905e-04, -2.5753386e-04, ...,
         9.9923554e-06,  1.3470695e-03, -1.2041409e-03]], dtype=float32)

In [21]:
import numpy as np

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

is_pos_def(C1)

False

In [17]:
#B1 = cp.asnumpy(s[0] * u[-1, :].reshape(n, n))
#C1 = cp.asnumpy(v[:, -1].reshape(m, m))

#np.save("B1_small_nn.npy", B1)
#np.save("C1_small_nn.npy", C1)

In [ ]:
is_pos_def(C1)

In [22]:
is_pos_def(B1)

False

In [ ]:
alpha = 0.0

С_new = (1 - alpha)*C1  + alpha*np.eye(len(np.diag(C1)))

while (not is_pos_def(С_new)):
    alpha += 0.01
    С_new = (1 - alpha)*C1  + alpha*np.eye(len(np.diag(C1)))

alpha

In [ ]:
alpha = 0.0

B_new = (1 - alpha)*B1  + alpha*np.eye(len(np.diag(B1)))

while (not is_pos_def(B_new)):
    alpha += 0.05
    B_new = (1 - alpha)*B1  + alpha*np.eye(len(np.diag(B1)))

alpha

In [ ]:
is_pos_def(С_new)

True

In [50]:
is_pos_def(B_new)

True

In [46]:
Bnew_square = np.linalg.cholesky(B_new)
Cnew_square = np.linalg.cholesky(С_new)

In [20]:
C1_square = np.linalg.cholesky(C1)

In [47]:
np.save("Bnew_square.npy", Bnew_square)
np.save("Cnew_square.npy", Cnew_square)

In [13]:
B1 = np.load("B1_small_nn.npy")
C1 = np.load("C1_small_nn.npy")

In [20]:
matrix1 = np.kron(B1, C1)
matrix1.shape

(147456, 147456)

In [9]:
grad_vectors.shape

torch.Size([625, 147456])

In [ ]:
import numpy as np
import pandas as pd
import torch
from scipy.linalg import kron
from scipy.sparse.linalg import svds

import pickle
with open("list_of_grads", "rb") as fp:   #Pickling
    list_of_grads = pickle.load(fp)

list_of_grads = torch.stack(list_of_grads, dim=0)

grad_vectors = torch.stack([grad.T.reshape(-1) for grad in list_of_grads])

import numpy as np
matrix2 = np.zeros((147456, 147456), dtype=float)
for ind, vector in enumerate(grad_vectors):
    print (ind, flush = True)
    for i, ielem in enumerate(vector):
        if (i%500 == 0):
            print (i,  flush = True)
        for j in range(i, len(vector)):
            matrix2[i][j] += ielem*jelem
            matrix2[j][i] += ielem*jelem

np.save(matrix2, "matrix2.npy")

0


In [ ]:
matrix2.shape